In [2]:
import time
import psycopg2

TPCH_QUERIES = {
    "Q1": """
        SELECT
          l_returnflag,
          l_linestatus,
          SUM(l_quantity)                                        AS sum_qty,
          SUM(l_extendedprice)                                   AS sum_base_price,
          SUM(l_extendedprice * (1 - l_discount))                AS sum_disc_price,
          SUM(l_extendedprice * (1 - l_discount) * (1 + l_tax))  AS sum_charge,
          AVG(l_quantity)                                        AS avg_qty,
          AVG(l_extendedprice)                                   AS avg_price,
          AVG(l_discount)                                        AS avg_disc,
          COUNT(*)                                               AS count_order
        FROM lineitem
        WHERE l_shipdate <= DATE '1998-12-01' - INTERVAL '90 days'
        GROUP BY l_returnflag, l_linestatus
        ORDER BY l_returnflag, l_linestatus
    """,

    "Q2": """
        SELECT
          s.s_acctbal,
          s.s_name,
          n.n_name,
          p.p_partkey,
          p.p_mfgr,
          s.s_address,
          s.s_phone,
          s.s_comment
        FROM part p
        JOIN partsupp ps  ON p.p_partkey = ps.ps_partkey
        JOIN supplier s   ON s.s_suppkey = ps.ps_suppkey
        JOIN nation n     ON s.s_nationkey = n.n_nationkey
        JOIN region r     ON n.n_regionkey = r.r_regionkey
        WHERE
          p.p_size = 15
          AND p.p_type LIKE '%BRASS'
          AND r.r_name = 'EUROPE'
          AND ps.ps_supplycost = (
            SELECT MIN(ps2.ps_supplycost)
            FROM partsupp ps2
            JOIN supplier s2 ON s2.s_suppkey = ps2.ps_suppkey
            JOIN nation n2   ON s2.s_nationkey = n2.n_nationkey
            JOIN region r2   ON n2.n_regionkey = r2.r_regionkey
            WHERE
              ps2.ps_partkey = p.p_partkey
              AND r2.r_name = 'EUROPE'
          )
        ORDER BY s.s_acctbal DESC, n.n_name, s.s_name, p.p_partkey
        LIMIT 100
    """,

    "Q3": """
        SELECT
          o.o_orderkey,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue,
          o.o_orderdate,
          o.o_shippriority
        FROM customer c
        JOIN orders o   ON c.c_custkey = o.o_custkey
        JOIN lineitem l ON l.l_orderkey = o.o_orderkey
        WHERE
          c.c_mktsegment = 'BUILDING'
          AND o.o_orderdate < DATE '1995-03-15'
          AND l.l_shipdate > DATE '1995-03-15'
        GROUP BY o.o_orderkey, o.o_orderdate, o.o_shippriority
        ORDER BY revenue DESC, o.o_orderdate
        LIMIT 10
    """,

    "Q4": """
        SELECT
          o.o_orderpriority,
          COUNT(*) AS order_count
        FROM orders o
        WHERE
          o.o_orderdate >= DATE '1993-07-01'
          AND o.o_orderdate <  DATE '1993-10-01'
          AND EXISTS (
            SELECT 1
            FROM lineitem l
            WHERE l.l_orderkey = o.o_orderkey
              AND l.l_commitdate < l.l_receiptdate
          )
        GROUP BY o.o_orderpriority
        ORDER BY o.o_orderpriority
    """,

    "Q5": """
        SELECT
          n.n_name,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue
        FROM customer c
        JOIN orders o   ON c.c_custkey = o.o_custkey
        JOIN lineitem l ON l.l_orderkey = o.o_orderkey
        JOIN supplier s ON s.s_suppkey = l.l_suppkey
        JOIN nation n   ON s.s_nationkey = n.n_nationkey
        JOIN region r   ON n.n_regionkey = r.r_regionkey
        WHERE
          r.r_name = 'ASIA'
          AND o.o_orderdate >= DATE '1994-01-01'
          AND o.o_orderdate <  DATE '1995-01-01'
        GROUP BY n.n_name
        ORDER BY revenue DESC
    """,

    "Q6": """
        SELECT
          SUM(l_extendedprice * l_discount) AS revenue
        FROM lineitem
        WHERE
          l_shipdate >= DATE '1994-01-01'
          AND l_shipdate <  DATE '1995-01-01'
          AND l_discount BETWEEN 0.05 AND 0.07
          AND l_quantity < 24
    """,

    "Q7": """
        SELECT
          n1.n_name AS supp_nation,
          n2.n_name AS cust_nation,
          EXTRACT(YEAR FROM l.l_shipdate) AS l_year,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue
        FROM supplier s
        JOIN lineitem l ON s.s_suppkey = l.l_suppkey
        JOIN orders o   ON o.o_orderkey = l.l_orderkey
        JOIN customer c ON c.c_custkey = o.o_custkey
        JOIN nation n1  ON s.s_nationkey = n1.n_nationkey
        JOIN nation n2  ON c.c_nationkey = n2.n_nationkey
        WHERE
          (
            (n1.n_name = 'FRANCE'  AND n2.n_name = 'GERMANY') OR
            (n1.n_name = 'GERMANY' AND n2.n_name = 'FRANCE')
          )
          AND l.l_shipdate >= DATE '1995-01-01'
          AND l.l_shipdate <= DATE '1996-12-31'
        GROUP BY n1.n_name, n2.n_name, EXTRACT(YEAR FROM l.l_shipdate)
        ORDER BY supp_nation, cust_nation, l_year
    """,

    "Q8": """
        SELECT
          EXTRACT(YEAR FROM o.o_orderdate) AS o_year,
          SUM(
            CASE
              WHEN n2.n_name = 'BRAZIL'
              THEN l.l_extendedprice * (1 - l.l_discount)
              ELSE 0
            END
          ) / SUM(l.l_extendedprice * (1 - l.l_discount)) AS mkt_share
        FROM part p
        JOIN lineitem l ON p.p_partkey = l.l_partkey
        JOIN orders o   ON o.o_orderkey = l.l_orderkey
        JOIN customer c ON c.c_custkey = o.o_custkey
        JOIN nation n1  ON c.c_nationkey = n1.n_nationkey
        JOIN region r   ON n1.n_regionkey = r.r_regionkey
        JOIN supplier s ON s.s_suppkey = l.l_suppkey
        JOIN nation n2  ON s.s_nationkey = n2.n_nationkey
        WHERE
          r.r_name = 'AMERICA'
          AND o.o_orderdate >= DATE '1995-01-01'
          AND o.o_orderdate <= DATE '1996-12-31'
          AND p.p_type = 'ECONOMY ANODIZED STEEL'
        GROUP BY EXTRACT(YEAR FROM o.o_orderdate)
        ORDER BY o_year
    """,

    "Q9": """
        SELECT
          n.n_name,
          EXTRACT(YEAR FROM o.o_orderdate) AS o_year,
          SUM(
            l.l_extendedprice * (1 - l.l_discount)
            - ps.ps_supplycost * l.l_quantity
          ) AS sum_profit
        FROM lineitem l
        JOIN orders o    ON o.o_orderkey = l.l_orderkey
        JOIN part p      ON p.p_partkey = l.l_partkey
        JOIN supplier s  ON s.s_suppkey = l.l_suppkey
        JOIN partsupp ps ON p.p_partkey = ps.ps_partkey
                        AND s.s_suppkey = ps.ps_suppkey
        JOIN nation n    ON s.s_nationkey = n.n_nationkey
        WHERE
          p.p_name LIKE '%green%'
        GROUP BY n.n_name, EXTRACT(YEAR FROM o.o_orderdate)
        ORDER BY n.n_name, o_year DESC
    """,

    "Q10": """
        SELECT
          c.c_custkey,
          c.c_name,
          SUM(l.l_extendedprice * (1 - l.l_discount)) AS revenue,
          c.c_acctbal,
          n.n_name,
          c.c_address,
          c.c_phone,
          c.c_comment
        FROM customer c
        JOIN orders o   ON c.c_custkey = o.o_custkey
        JOIN lineitem l ON l.l_orderkey = o.o_orderkey
        JOIN nation n   ON c.c_nationkey = n.n_nationkey
        WHERE
          o.o_orderdate >= DATE '1993-10-01'
          AND o.o_orderdate <  DATE '1994-01-01'
          AND l.l_returnflag = 'R'
        GROUP BY
          c.c_custkey,
          c.c_name,
          c.c_acctbal,
          n.n_name,
          c.c_address,
          c.c_phone,
          c.c_comment
        ORDER BY revenue DESC
        LIMIT 20
    """
}



def run_tpch_warm_only(repeats: int = 3):
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="tpch",
        user="bench_user",
        password="940411",
    )

    results = []

    try:
        with conn:
            with conn.cursor() as cur:
                for name, sql in TPCH_QUERIES.items():
                    times = []
                    print(f"\n--- {name} ---")
                    for i in range(repeats):
                        start = time.perf_counter()
                        cur.execute(sql)
                        cur.fetchall() 
                        elapsed_ms = (time.perf_counter() - start) * 1000
                        times.append(elapsed_ms)
                        print(f"{name} run {i+1}: {elapsed_ms:.1f} ms")
                    warm_min = min(times)
                    warm_avg = sum(times) / len(times)
                    print(f"{name}: warm_min={warm_min:.1f} ms, warm_avg={warm_avg:.1f} ms")
                    results.append((name, warm_min, warm_avg))
    finally:
        conn.close()

    print("\n=== Summary (PostgreSQL TPCH, warm only) ===")
    print("query,warm_min_ms,warm_avg_ms")
    for name, warm_min, warm_avg in results:
        print(f"{name},{warm_min:.1f},{warm_avg:.1f}")


if __name__ == "__main__":
    run_tpch_warm_only(repeats=3)



--- Q1 ---
Q1 run 1: 6678.6 ms
Q1 run 2: 6589.3 ms
Q1 run 3: 6578.8 ms
Q1: warm_min=6578.8 ms, warm_avg=6615.5 ms

--- Q2 ---
Q2 run 1: 2756.7 ms
Q2 run 2: 2787.0 ms
Q2 run 3: 2701.8 ms
Q2: warm_min=2701.8 ms, warm_avg=2748.5 ms

--- Q3 ---
Q3 run 1: 3153.3 ms
Q3 run 2: 3279.4 ms
Q3 run 3: 3216.4 ms
Q3: warm_min=3153.3 ms, warm_avg=3216.4 ms

--- Q4 ---
Q4 run 1: 2958.7 ms
Q4 run 2: 3044.7 ms
Q4 run 3: 3027.3 ms
Q4: warm_min=2958.7 ms, warm_avg=3010.2 ms

--- Q5 ---
Q5 run 1: 3219.7 ms
Q5 run 2: 3342.2 ms
Q5 run 3: 3314.2 ms
Q5: warm_min=3219.7 ms, warm_avg=3292.1 ms

--- Q6 ---
Q6 run 1: 1661.7 ms
Q6 run 2: 1707.0 ms
Q6 run 3: 1466.0 ms
Q6: warm_min=1466.0 ms, warm_avg=1611.6 ms

--- Q7 ---
Q7 run 1: 2381.1 ms
Q7 run 2: 2441.7 ms
Q7 run 3: 2256.2 ms
Q7: warm_min=2256.2 ms, warm_avg=2359.7 ms

--- Q8 ---
Q8 run 1: 2998.6 ms
Q8 run 2: 2997.3 ms
Q8 run 3: 2847.6 ms
Q8: warm_min=2847.6 ms, warm_avg=2947.9 ms

--- Q9 ---
Q9 run 1: 8228.6 ms
Q9 run 2: 8342.2 ms
Q9 run 3: 8185.7 ms
Q9: warm